# Core

> The Plash CLI tool

In [ ]:
#| default_exp core

In [ ]:
#| export
from fastcore.all import *
from fastcore.xdg import *
import secrets, webbrowser, json, httpx, io, tarfile
from pathlib import Path
from uuid import uuid4
from time import time, sleep
import io, os, re, tarfile, tomllib

from plash_cli import __version__

In [ ]:
#| hide
from tempfile import TemporaryDirectory

## Helpers

In [ ]:
#| export
PLASH_CONFIG_HOME = xdg_config_home() / 'plash_config.json'
PLASH_DOMAIN = os.getenv("PLASH_DOMAIN","pla.sh")  # pla.sh plash-dev.answer.ai localhost:5002

In [ ]:
#| export
def get_client(cookie_file):
    client = httpx.Client()
    if not cookie_file.exists():
        raise FileNotFoundError("Plash config not found. Please run plash_login and try again.")
    cookies = Path(cookie_file).read_json()
    client.cookies.update(cookies)
    client.headers.update({'X-PLASH': 'true', 'User-Agent': f'plash_cli/{__version__}'})
    return client

In [ ]:
#| export
def mk_auth_req(url:str, method:str='get', **kwargs): return getattr(get_client(PLASH_CONFIG_HOME), method)(url, **kwargs)

In [ ]:
#| export
def get_app_name(path:Path):
    plash_app = Path(path) / '.plash'
    if not plash_app.exists(): raise FileNotFoundError(f"File not found: {plash_app=}")
    env = parse_env(fn=plash_app)
    if name:=env.get("PLASH_APP_NAME"): return name
    if aid :=env.get('PLASH_APP_ID'): 
            plash_app.write_text(f"export PLASH_APP_NAME={aid}")
            return aid
    raise RuntimeError(f"{plash_app=} did not have a PLASH_APP_NAME")


In [ ]:
#| export
def endpoint(sub='', rt=''):
    p = "http" if "localhost" in PLASH_DOMAIN else "https"
    return f"{p}://{sub}{'.' if sub else ''}{PLASH_DOMAIN}{rt}"

In [ ]:
#| export
def is_included(path):
    "Returns True if path should be included in deployment"
    if path.name.startswith('.'): return False
    if path.suffix == '.pyc': return False
    excludes = {'.git', '__pycache__', '.gitignore', '.env', 
                '.pytest_cache', '.venv', 'venv', '.ipynb_checkpoints',
                '.vscode', '.idea', '.sesskey'}
    return not any(p in excludes for p in path.parts)

## Plash - login

In [ ]:
#| export
def poll_cookies(paircode, interval=1, timeout=180):
    "Poll server for token until received or timeout"
    start = time()
    client = httpx.Client()
    url = endpoint(rt=f"/cli_token?paircode={paircode}")
    while time()-start < timeout:
        resp = client.get(url).raise_for_status()
        if resp.text.strip(): return dict(client.cookies)
        sleep(interval)
        
@call_parse
def login():
    "Authenticate CLI with server and save config"
    paircode = secrets.token_urlsafe(16)
    login_url = httpx.get(endpoint(rt=f"/cli_login?paircode={paircode}")).text
    print(f"Opening browser for authentication:\n{login_url}\n")
    webbrowser.open(login_url)
    
    cookies = poll_cookies(paircode)
    if cookies:
        Path(PLASH_CONFIG_HOME).write_text(json.dumps(cookies))
        print(f"Authentication successful! Config saved to {PLASH_CONFIG_HOME}")
    else: print("Authentication timed out.")

## App - deploy

Dependencies can be provided via a requirements.txt file or with inline dependencies following PEP 723.

In [ ]:
#| export
pat = r'(?m)^# /// (?P<type>[a-zA-Z0-9-]+)$\s(?P<content>(^#(| .*)$\s)+)^# ///$'

def _deps(script: bytes | str) -> dict | None:
    'Get the dependencies from the script. From: https://peps.python.org/pep-0723/'
    name = 'script'
    if isinstance(script, bytes): script = script.decode('utf-8')
    matches = L(re.finditer(pat, script)).filter(lambda m: m.group('type') == name)
    if len(matches) > 1: raise ValueError(f'Multiple {name} blocks found')
    elif len(matches) == 1:
        content = ''.join(line[2:] if line.startswith('# ') else line[1:]
                          for line in matches[0].group('content').splitlines(keepends=True))
        return '\n'.join(tomllib.loads(content)['dependencies'])
    else: return None

Lets test on some demo apps:

In [ ]:
test_eq(_deps(Path("../examples/script_app/main.py").read_text()),"python-fasthtml")
test_is(_deps(Path("../examples/minimal/main.py").read_text()),None)

In [ ]:
#| export
class PlashError(Exception): pass

def validate_app(path):
    "Validates directory `path` is a deployable Plash app"
    if not (path / 'main.py').exists():
        raise PlashError('A Plash app requires a main.py file.')
    deps = _deps((path / 'main.py').read_text())
    if  deps and (path/"requirements.txt").exists(): 
        raise PlashError('A Plash app should not contain both a requirements.txt file and inline dependencies (see PEP723).')

All test apps should be valid:

In [ ]:
for d in Path("../examples/").iterdir(): test_is(validate_app(d),None)

Lets test each failure case:

In [ ]:
with TemporaryDirectory() as td:
    td = Path(td)
    test_fail(validate_app, args=(td,), contains="main.py")
    
    # test failure case of deps in both main.py and requirements.txt
    (td / "requirements.txt").write_text("")
    (td / "main.py").write_text(Path("../examples/script_app/main.py").read_text())
    test_fail(validate_app, args=(td,), contains="not contain both")

In [ ]:
#| export
def create_tar_archive(path:Path) -> tuple[io.BytesIO, int]:
    "Creates a tar archive of a directory, excluding files based on is_included"
    tarz = io.BytesIO()
    files = L(path if path.is_file() else Path(path).iterdir()).filter(is_included)
    with tarfile.open(fileobj=tarz, mode='w:gz') as tar:
        for f in files: tar.add(f, arcname=f.name)
        if deps:=_deps((path / 'main.py').read_bytes()):
            info = tarfile.TarInfo('requirements.txt')
            info.size = len(deps)
            tar.addfile(info, io.BytesIO(deps.encode('utf-8')))
    tarz.seek(0)
    return tarz, len(files)

In [ ]:
#| export
@call_parse
def deploy(
    path:Path=Path('.'),    # Path to project
    name:str=None,          # Overrides the .plash file in project root if provided
    force_data:bool=False): # Overwrite data/ directory during deployment
    "Deploy app to production, ignores paths starting with '.', excludes data/ directory by default unless --force_data is used."
    print('Initializing deployment...')
    if name == '': print('Error: App name cannot be an empty string'); return
    if not path.is_dir(): print("Error: Path should point to the project directory"); return
    try: validate_app(path)
    except PlashError as e: print(f"Error: {str(e)}\nInvalid path: {path}"); return
    
    try: 
        if not name: name = get_app_name(path)
    except FileNotFoundError:
        plash_app = path / '.plash'
        name = f'fasthtml-app-{str(uuid4())[:8]}'
        plash_app.write_text(f'export PLASH_APP_NAME={name}')
    
    tarz, _ = create_tar_archive(path)
    resp = mk_auth_req(endpoint(rt="/upload"), "post", files={'file': tarz}, timeout=300.0, 
                       data={'name': name, 'force_data': force_data})
    if resp.status_code == 200:
        print('✅ Upload complete! Your app is currently being built.')
        print(f'It will be live at {name if '.' in name else endpoint(sub=name)}')
    else: print(f'Failure: {resp.status_code}\n{resp.text}')

## App - view

In [ ]:
#| export
@call_parse
def view(
    path:Path=Path('.'), # Path to project directory
    name:str=None,     # Overrides the .plash file in project root if provided
):
    "Open your app in the browser"
    if not name: name = get_app_name(path)
    url = name if '.' in name else endpoint(sub=name)
    print(f"Opening browser to view app :\n{url}\n")
    webbrowser.open(url)

## App - delete

In [ ]:
#| export
@call_parse
def delete(
    path:Path=Path('.'), # Path to project
    name:str=None,     # Overrides the .plash file in project root if provided
    force:bool=False):   # Skip confirmation prompt
    'Delete your deployed app'
    if not name: name = get_app_name(path)
    if not force:
        confirm = input(f"Are you sure you want to delete app '{name}'? This action cannot be undone. [y/N]: ")
        if confirm.lower() not in ['y', 'yes']:
            print("Deletion cancelled.")
            return
    
    print(f"Deleting app '{name}'...")
    r = mk_auth_req(endpoint(rt=f"/delete?name={name}"), "delete")
    return r.text

## App - start stop

In [ ]:
#| export
def endpoint_func(endpoint_name):
    'Creates a function for a specific API endpoint'
    def func(
        path:Path=Path('.'), # Path to project
        name:str=None,     # Overrides the .plash file in project root if provided
    ):
        if not name: name = get_app_name(path)
        r = mk_auth_req(endpoint(rt=f"{endpoint_name}?name={name}"))
        return r.text
    
    # Set the function name and docstring
    func.__name__ = endpoint_name
    func.__doc__ = f"Access the '{endpoint_name}' endpoint for your app"
    
    return call_parse(func)

# Create endpoint-specific functions
stop = endpoint_func('/stop')
start = endpoint_func('/start')

## App - logs

In [ ]:
#| export
log_modes = str_enum('log_modes', 'build', 'app')

In [ ]:
#| export
@call_parse
def logs(
    path:Path=Path('.'),    # Path to project
    name:str=None,          # Overrides the .plash file in project root if provided
    mode:log_modes='build', # Choose between build or app logs
    tail:bool=False):       # Tail the logs
    'Prints the logs for your deployed app'
    if not name: name = get_app_name(path)
    if tail:
        text = ''
        while True:
            try:
                r = mk_auth_req(endpoint(rt=f"/logs?name={name}&mode={mode}"))
                if r.status_code == 200:
                    print(r.text[len(text):], end='') # Only print updates
                    text = r.text
                    if mode == 'build' and 'Build End Time:' in r.text: break
                    sleep(1)
                else:
                    print(f"Error: {r.status_code}")
            except KeyboardInterrupt:
                return "\nExiting"
    r = mk_auth_req(endpoint(rt=f"/logs?name={name}&mode={mode}"))
    return r.text

## App - download

In [ ]:
#| export
@call_parse
def download(
    path:Path=Path('.'),                 # Path to project
    name:str=None,                       # Overrides the .plash file in project root if provided
    save_path:Path=Path("./download/")): # Save path (optional)
    'Download your deployed app'
    if not name: name = get_app_name(path)
    try: save_path.mkdir(exist_ok=False)
    except: print(f"ERROR: Save path ({save_path}) already exists. Please rename or delete this folder to avoid accidental overwrites.")
    else:
        response = mk_auth_req(endpoint(rt=f'/download?name={name}')).raise_for_status()
        file_bytes = io.BytesIO(response.content)
        with tarfile.open(fileobj=file_bytes, mode="r:gz") as tar: tar.extractall(path=save_path)
        print(f"Downloaded your app to: {save_path}")

## List Apps

In [ ]:
#|export
@call_parse
def apps(verbose:bool=False):
    "List your deployed apps (verbose shows status table: 1=running, 0=stopped)"
    r = mk_auth_req(endpoint(rt="/user_apps")).raise_for_status()
    apps = r.json()
    if not apps: return "You don't have any deployed Plash apps."
    if verbose: [print(f'{a['running']} {a['name']}') for a in apps]
    else: [print(a['name']) for a in apps]

## Export

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()